<a href="https://colab.research.google.com/github/sarath140/Video2VideoTranslation_EngIndic/blob/main/Copy_of_Fnl_Trans2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AI4Bharat/IndicTrans2
%cd IndicTrans2/huggingface_interface

Cloning into 'IndicTrans2'...
remote: Enumerating objects: 767, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 767 (delta 151), reused 105 (delta 104), pack-reused 585 (from 3)
Receiving objects: 100% (767/767), 4.16 MiB | 7.93 MiB/s, done.
Resolving deltas: 100% (493/493), done.
/content/IndicTrans2/huggingface_interface


In [2]:
!bash install.sh

Setting up the environment in the /content/IndicTrans2/huggingface_interface
Creating a virtual environment with python3
install.sh: line 10: conda: command not found
install.sh: line 11: conda: command not found
Installing all the dependencies
install.sh: line 14: conda: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  DEPRECATION: Building 'mosestokenizer' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'mosestokenizer'. 

In [3]:
import os
import torch
from pathlib import Path
from google.colab import drive
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.IndicTransToolkit import IndicProcessor

# ========================
# Utility Functions
# ========================

def format_timestamp(seconds: float) -> str:
    """Preserve SRT timestamp format."""
    millis = int((seconds - int(seconds)) * 1000)
    seconds = int(seconds)
    mins, sec = divmod(seconds, 60)
    hrs, mins = divmod(mins, 60)
    return f"{hrs:02d}:{mins:02d}:{sec:02d},{millis:03d}"

# ========================
# Environment Setup
# ========================

drive.mount('/content/drive')

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang = "eng_Latn"
tgt_lang = "hin_Deva"
model_name = "ai4bharat/indictrans2-en-indic-1B"

# Load model + tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

# Input / Output paths
input_dir = "/content/drive/MyDrive/Commercial Horticulture English ASR"
output_dir = "/content/drive/MyDrive/Commercial Horticulture English MThi"

# ========================
# Translation Logic
# ========================

def translate_srt(input_path, output_srt, output_txt):
    """Translate a single SRT file and save both .srt and .txt outputs."""
    with open(input_path, 'r', encoding='utf-8') as f:
        srt_lines = f.readlines()

    translated_srt, translated_txt = [], []

    for i, line in enumerate(srt_lines):
        if i % 4 == 2:  # Subtitle line
            text = line.strip()
            if text:
                batch = ip.preprocess_batch([text], src_lang=src_lang, tgt_lang=tgt_lang)
                inputs = tokenizer(batch, truncation=True, padding="longest", return_tensors="pt").to(DEVICE)

                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_length=256,
                        num_beams=5,
                        num_return_sequences=1,
                    )

                decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
                final_text = ip.postprocess_batch([decoded], lang=tgt_lang)[0]

                translated_srt.append(final_text)
                translated_txt.append(final_text)
        else:
            translated_srt.append(line.strip())

    # Save outputs
    with open(output_srt, 'w', encoding='utf-8') as f:
        f.write("\n".join(translated_srt))

    with open(output_txt, 'w', encoding='utf-8') as f:
        f.write("\n".join(translated_txt))

    print(f"✅ Saved: {output_srt}")
    print(f"✅ Saved: {output_txt}")

# ========================
# Batch Processing
# ========================

for root, _, files in os.walk(input_dir):
    for file in files:
        if file.endswith(".srt"):
            input_path = os.path.join(root, file)
            relative = os.path.relpath(root, input_dir)
            save_dir = os.path.join(output_dir, relative)
            os.makedirs(save_dir, exist_ok=True)

            output_srt = os.path.join(save_dir, file)
            output_txt = os.path.join(save_dir, file.replace(".srt", ".txt"))

            print(f"🔄 Translating: {input_path}")
            translate_srt(input_path, output_srt, output_txt)


Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenization_indictrans.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

modeling_indictrans.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

🔄 Translating: /content/drive/MyDrive/Commercial Horticulture English ASR/Chapter 4 - Diseases and Control/Chapter 4 - Responsible use of Pesticides and Natural Alternatives.srt
✅ Translated SRT saved to: /content/drive/MyDrive/Commercial Horticulture English MThi/Chapter 4 - Diseases and Control/Chapter 4 - Responsible use of Pesticides and Natural Alternatives.srt
✅ Translated TXT saved to: /content/drive/MyDrive/Commercial Horticulture English MThi/Chapter 4 - Diseases and Control/Chapter 4 - Responsible use of Pesticides and Natural Alternatives.txt
🔄 Translating: /content/drive/MyDrive/Commercial Horticulture English ASR/Chapter 4 - Diseases and Control/Chapter 4B - Organic methods like companion planting and natural predators.srt
✅ Translated SRT saved to: /content/drive/MyDrive/Commercial Horticulture English MThi/Chapter 4 - Diseases and Control/Chapter 4B - Organic methods like companion planting and natural predators.srt
✅ Translated TXT saved to: /content/drive/MyDrive/Comme